<a href="https://colab.research.google.com/github/bellajelly/ML-week3-dimensionality-reduction/blob/main/dimensionality_reduction_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dimensionality Reduction Analysis of C. elegans Gene Regulatory Networks

**Author:** Bell Jelly  
**Course:** Machine Learning - Week 3  
**Date:** February 2026

## Project Overview

This notebook explores dimensionality reduction techniques (PCA and t-SNE) applied to a physical gene regulatory network from *C. elegans*. We investigate two complementary approaches:

1. **Option A - Regulatory Target Analysis**: Clustering transcription factors based on which genes they regulate
2. **Option B - Network Topology Analysis**: Clustering genes based on their structural properties in the network

By comparing these approaches, we explore what patterns emerge and what information might be lost through dimensionality reduction.

## 1. Setup and Data Loading

First, we'll import necessary libraries and load the C. elegans gene regulatory network data.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 8)

In [ ]:
# Load data from GitHub repository
# You'll need to replace this URL with your actual raw GitHub file URLs
github_base = 'https://raw.githubusercontent.com/bellajelly/week3-dimensionality-reduction/main/data/'

# Load the gene regulatory network
network_df = pd.read_csv(github_base + 'TableS3_WT_functional_priors.csv', on_bad_lines='skip')

# Load dataset metadata
metadata_df = pd.read_csv(github_base + 'TableS1_datasets_for_prior.csv')

print(f"Network shape: {network_df.shape}")
print(f"Unique regulators: {network_df['regulator'].nunique()}")
print(f"Unique targets: {network_df['target'].nunique()}")
print(f"\nFirst few interactions:")
network_df.head()

## 2. Option A - Regulatory Target Matrix Analysis

### What are we doing?

We're creating a matrix where:
- **Rows** = Transcription factors (regulators)
- **Columns** = Target genes
- **Values** = 1 if the TF regulates that target, 0 otherwise

This will be a 289 × 19,185 matrix. By applying PCA and t-SNE, we can see which transcription factors have similar regulatory programs.

In [ ]:
# Create the regulatory target matrix
# This creates a binary matrix: 1 if regulator controls target, 0 otherwise

# Get unique regulators and targets
regulators = network_df['regulator'].unique()
targets = network_df['target'].unique()

print(f"Creating matrix with {len(regulators)} regulators and {len(targets)} targets...")

# Initialize empty matrix
reg_target_matrix = pd.DataFrame(0, index=regulators, columns=targets)

# Fill in the matrix with regulatory relationships
for _, row in network_df.iterrows():
    reg_target_matrix.loc[row['regulator'], row['target']] = 1

print(f"Matrix shape: {reg_target_matrix.shape}")
print(f"Sparsity: {(reg_target_matrix == 0).sum().sum() / (reg_target_matrix.shape[0] * reg_target_matrix.shape[1]) * 100:.2f}% zeros")

# Convert to numpy array for sklearn
X_targets = reg_target_matrix.values
regulator_names = reg_target_matrix.index.tolist()

### Understanding the transformation

**Question for you to think about:** This matrix is very sparse (mostly zeros). What does that tell you about gene regulation in C. elegans? Are transcription factors specialists or generalists?

### 2.1 Apply PCA to Regulatory Target Matrix

**What PCA does:**
- Finds the directions (principal components) of maximum variance in our 19,185-dimensional space
- PC1 captures the biggest difference between transcription factors
- PC2 captures the second biggest difference (perpendicular to PC1)
- We can visualize high-dimensional data in 2D while preserving the main patterns

In [ ]:
# Standardize the data (important for PCA)
# This ensures all features contribute equally
scaler_targets = StandardScaler()
X_targets_scaled = scaler_targets.fit_transform(X_targets)

# Apply PCA
pca_targets = PCA(n_components=2)
X_targets_pca = pca_targets.fit_transform(X_targets_scaled)

# Print explained variance
print("Explained variance ratio:")
print(f"PC1: {pca_targets.explained_variance_ratio_[0]:.4f} ({pca_targets.explained_variance_ratio_[0]*100:.2f}%)")
print(f"PC2: {pca_targets.explained_variance_ratio_[1]:.4f} ({pca_targets.explained_variance_ratio_[1]*100:.2f}%)")
print(f"Total variance explained: {pca_targets.explained_variance_ratio_.sum()*100:.2f}%")

**Insight from explained variance:**

The explained variance tells us how much information we're retaining. If PC1 and PC2 together explain only 20% of variance, that means 80% of the variation in regulatory targets is lost when we reduce to 2D. This is important for our ethical reflection - what are we missing?

In [ ]:
# Visualize PCA results
plt.figure(figsize=(12, 8))
plt.scatter(X_targets_pca[:, 0], X_targets_pca[:, 1], alpha=0.6, s=50)
plt.xlabel(f'PC1 ({pca_targets.explained_variance_ratio_[0]*100:.2f}% variance)')
plt.ylabel(f'PC2 ({pca_targets.explained_variance_ratio_[1]*100:.2f}% variance)')
plt.title('PCA: Transcription Factors Clustered by Regulatory Targets')
plt.grid(True, alpha=0.3)

# Optionally, label some interesting points (like lin-42 if it exists)
if 'lin-42' in regulator_names:
    idx = regulator_names.index('lin-42')
    plt.scatter(X_targets_pca[idx, 0], X_targets_pca[idx, 1],
                color='red', s=200, marker='*', label='lin-42 (clock gene)')
    plt.legend()

plt.tight_layout()
plt.show()

### 2.2 Apply t-SNE to Regulatory Target Matrix

**What t-SNE does:**
- Preserves local neighborhood structure (keeps similar points together)
- Non-linear method - can reveal complex clustering patterns
- Good at showing distinct groups, but distances between clusters are not meaningful
- Perplexity parameter controls how many neighbors each point considers (typically 5-50)

In [ ]:
# Apply t-SNE
# Note: t-SNE can be slow for large datasets
tsne_targets = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
X_targets_tsne = tsne_targets.fit_transform(X_targets_scaled)

print("t-SNE transformation complete!")

In [ ]:
# Visualize t-SNE results
plt.figure(figsize=(12, 8))
plt.scatter(X_targets_tsne[:, 0], X_targets_tsne[:, 1], alpha=0.6, s=50)
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.title('t-SNE: Transcription Factors Clustered by Regulatory Targets')
plt.grid(True, alpha=0.3)

# Label lin-42 if it exists
if 'lin-42' in regulator_names:
    idx = regulator_names.index('lin-42')
    plt.scatter(X_targets_tsne[idx, 0], X_targets_tsne[idx, 1],
                color='red', s=200, marker='*', label='lin-42 (clock gene)')
    plt.legend()

plt.tight_layout()
plt.show()

### 2.3 Compare PCA vs t-SNE for Regulatory Targets

**Visual comparison:** Do you see different clustering patterns? Are the same transcription factors grouped together in both methods?

In [ ]:
# Side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# PCA plot
axes[0].scatter(X_targets_pca[:, 0], X_targets_pca[:, 1], alpha=0.6, s=50)
axes[0].set_xlabel(f'PC1 ({pca_targets.explained_variance_ratio_[0]*100:.2f}%)')
axes[0].set_ylabel(f'PC2 ({pca_targets.explained_variance_ratio_[1]*100:.2f}%)')
axes[0].set_title('PCA: Linear Projection')
axes[0].grid(True, alpha=0.3)

# t-SNE plot
axes[1].scatter(X_targets_tsne[:, 0], X_targets_tsne[:, 1], alpha=0.6, s=50)
axes[1].set_xlabel('t-SNE Dimension 1')
axes[1].set_ylabel('t-SNE Dimension 2')
axes[1].set_title('t-SNE: Non-linear Embedding')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Option B - Network Topology Analysis

### What are we doing?

Instead of looking at *which* genes are regulated, we look at *how* genes are connected in the network structure. We'll calculate network features for each gene:
- **Out-degree**: How many genes does this gene regulate?
- **In-degree**: How many genes regulate this gene?
- **Betweenness centrality**: How often is this gene on paths between other genes?
- **Clustering coefficient**: Are this gene's neighbors also connected to each other?
- **PageRank**: How "important" is this gene based on its connections?

This gives us a much smaller feature space (5 features instead of 19,185), but captures different information about network structure.

In [ ]:
# Build a directed graph from the network data
G = nx.DiGraph()

# Add edges (regulatory relationships)
for _, row in network_df.iterrows():
    G.add_edge(row['regulator'], row['target'])

print(f"Network created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

In [ ]:
# Calculate network features for each gene
print("Calculating network features...")

# Out-degree: number of genes this gene regulates
out_degree = dict(G.out_degree())

# In-degree: number of genes that regulate this gene
in_degree = dict(G.in_degree())

# Betweenness centrality: how often gene lies on shortest paths
betweenness = nx.betweenness_centrality(G)

# Clustering coefficient: are neighbors connected to each other?
clustering = nx.clustering(G.to_undirected())  # Convert to undirected for clustering

# PageRank: importance based on connections
pagerank = nx.pagerank(G)

print("Network features calculated!")

In [ ]:
# Create network features dataframe
network_features = pd.DataFrame({
    'gene': list(G.nodes()),
    'out_degree': [out_degree[node] for node in G.nodes()],
    'in_degree': [in_degree[node] for node in G.nodes()],
    'betweenness': [betweenness[node] for node in G.nodes()],
    'clustering': [clustering[node] for node in G.nodes()],
    'pagerank': [pagerank[node] for node in G.nodes()]
})

print(f"Network features shape: {network_features.shape}")
print("\nSummary statistics:")
network_features.describe()

In [ ]:
# Check if lin-42 exists and show its features
if 'lin-42' in network_features['gene'].values:
    lin42_features = network_features[network_features['gene'] == 'lin-42']
    print("\nlin-42 (clock gene) network features:")
    print(lin42_features)

### 3.1 Apply PCA to Network Features

Now we'll apply PCA to this 5-dimensional feature space. Since we only have 5 features, we can keep both components and see what each captures.

In [ ]:
# Prepare data for PCA
X_network = network_features[['out_degree', 'in_degree', 'betweenness', 'clustering', 'pagerank']].values
gene_names = network_features['gene'].tolist()

# Standardize features
scaler_network = StandardScaler()
X_network_scaled = scaler_network.fit_transform(X_network)

# Apply PCA
pca_network = PCA(n_components=2)
X_network_pca = pca_network.fit_transform(X_network_scaled)

# Print explained variance
print("Explained variance ratio:")
print(f"PC1: {pca_network.explained_variance_ratio_[0]:.4f} ({pca_network.explained_variance_ratio_[0]*100:.2f}%)")
print(f"PC2: {pca_network.explained_variance_ratio_[1]:.4f} ({pca_network.explained_variance_ratio_[1]*100:.2f}%)")
print(f"Total variance explained: {pca_network.explained_variance_ratio_.sum()*100:.2f}%")

In [ ]:
# Visualize PCA results for network features
plt.figure(figsize=(12, 8))
plt.scatter(X_network_pca[:, 0], X_network_pca[:, 1], alpha=0.5, s=30)
plt.xlabel(f'PC1 ({pca_network.explained_variance_ratio_[0]*100:.2f}% variance)')
plt.ylabel(f'PC2 ({pca_network.explained_variance_ratio_[1]*100:.2f}% variance)')
plt.title('PCA: Genes Clustered by Network Topology Features')
plt.grid(True, alpha=0.3)

# Highlight lin-42 if it exists
if 'lin-42' in gene_names:
    idx = gene_names.index('lin-42')
    plt.scatter(X_network_pca[idx, 0], X_network_pca[idx, 1],
                color='red', s=200, marker='*', label='lin-42 (clock gene)')
    plt.legend()

plt.tight_layout()
plt.show()

### Understanding PC loadings

The "loadings" tell us which original features contribute most to each PC. This helps us interpret what PC1 and PC2 actually represent.

In [ ]:
# Display feature loadings
loadings = pd.DataFrame(
    pca_network.components_.T,
    columns=['PC1', 'PC2'],
    index=['out_degree', 'in_degree', 'betweenness', 'clustering', 'pagerank']
)

print("Feature loadings (how much each feature contributes to each PC):")
print(loadings)

# Visualize loadings
loadings.plot(kind='bar', figsize=(10, 6))
plt.title('PCA Feature Loadings for Network Topology')
plt.xlabel('Network Feature')
plt.ylabel('Loading Weight')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Interpretation question:** If PC1 has high positive loadings for out_degree and betweenness, what does that tell you about genes with high PC1 scores? (Hint: Think hub genes vs. peripheral genes)

### 3.2 Apply t-SNE to Network Features

In [ ]:
# Apply t-SNE
tsne_network = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
X_network_tsne = tsne_network.fit_transform(X_network_scaled)

print("t-SNE transformation complete!")

In [ ]:
# Visualize t-SNE results
plt.figure(figsize=(12, 8))
plt.scatter(X_network_tsne[:, 0], X_network_tsne[:, 1], alpha=0.5, s=30)
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.title('t-SNE: Genes Clustered by Network Topology Features')
plt.grid(True, alpha=0.3)

# Highlight lin-42 if it exists
if 'lin-42' in gene_names:
    idx = gene_names.index('lin-42')
    plt.scatter(X_network_tsne[idx, 0], X_network_tsne[idx, 1],
                color='red', s=200, marker='*', label='lin-42 (clock gene)')
    plt.legend()

plt.tight_layout()
plt.show()

### 3.3 Compare PCA vs t-SNE for Network Features

In [ ]:
# Side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# PCA plot
axes[0].scatter(X_network_pca[:, 0], X_network_pca[:, 1], alpha=0.5, s=30)
axes[0].set_xlabel(f'PC1 ({pca_network.explained_variance_ratio_[0]*100:.2f}%)')
axes[0].set_ylabel(f'PC2 ({pca_network.explained_variance_ratio_[1]*100:.2f}%)')
axes[0].set_title('PCA: Network Topology')
axes[0].grid(True, alpha=0.3)

# t-SNE plot
axes[1].scatter(X_network_tsne[:, 0], X_network_tsne[:, 1], alpha=0.5, s=30)
axes[1].set_xlabel('t-SNE Dimension 1')
axes[1].set_ylabel('t-SNE Dimension 2')
axes[1].set_title('t-SNE: Network Topology')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Grand Comparison: Option A vs Option B

Now let's compare what we learned from both approaches:

In [ ]:
# Create a 2x2 grid comparing all four visualizations
fig, axes = plt.subplots(2, 2, figsize=(18, 16))

# Option A - PCA
axes[0, 0].scatter(X_targets_pca[:, 0], X_targets_pca[:, 1], alpha=0.6, s=50)
axes[0, 0].set_xlabel(f'PC1 ({pca_targets.explained_variance_ratio_[0]*100:.2f}%)')
axes[0, 0].set_ylabel(f'PC2 ({pca_targets.explained_variance_ratio_[1]*100:.2f}%)')
axes[0, 0].set_title('Option A - PCA: Regulatory Targets')
axes[0, 0].grid(True, alpha=0.3)

# Option A - t-SNE
axes[0, 1].scatter(X_targets_tsne[:, 0], X_targets_tsne[:, 1], alpha=0.6, s=50)
axes[0, 1].set_xlabel('t-SNE Dim 1')
axes[0, 1].set_ylabel('t-SNE Dim 2')
axes[0, 1].set_title('Option A - t-SNE: Regulatory Targets')
axes[0, 1].grid(True, alpha=0.3)

# Option B - PCA
axes[1, 0].scatter(X_network_pca[:, 0], X_network_pca[:, 1], alpha=0.5, s=30, color='orange')
axes[1, 0].set_xlabel(f'PC1 ({pca_network.explained_variance_ratio_[0]*100:.2f}%)')
axes[1, 0].set_ylabel(f'PC2 ({pca_network.explained_variance_ratio_[1]*100:.2f}%)')
axes[1, 0].set_title('Option B - PCA: Network Topology')
axes[1, 0].grid(True, alpha=0.3)

# Option B - t-SNE
axes[1, 1].scatter(X_network_tsne[:, 0], X_network_tsne[:, 1], alpha=0.5, s=30, color='orange')
axes[1, 1].set_xlabel('t-SNE Dim 1')
axes[1, 1].set_ylabel('t-SNE Dim 2')
axes[1, 1].set_title('Option B - t-SNE: Network Topology')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Key Insights and Observations

**Questions to guide your analysis:**

1. **Explained variance**: How much information did we retain in each approach?
2. **Cluster structure**: Do you see distinct clusters? Are they similar between PCA and t-SNE?
3. **Biological meaning**: What might the clusters represent?
4. **Option A vs Option B**: Do they tell different stories? Can a gene be central in one view but peripheral in another?

*Add your observations here after running the analysis*

## 6. What Information Was Lost?

This is critical for your ethical reflection. Let's quantify what we lost through dimensionality reduction.

In [ ]:
# For Option A: Show cumulative explained variance
pca_full = PCA()
pca_full.fit(X_targets_scaled)

# Plot cumulative explained variance
plt.figure(figsize=(12, 6))
cumsum_var = np.cumsum(pca_full.explained_variance_ratio_)
plt.plot(range(1, len(cumsum_var) + 1), cumsum_var)
plt.axhline(y=0.95, color='r', linestyle='--', label='95% variance')
plt.axhline(y=0.90, color='orange', linestyle='--', label='90% variance')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('How Many Components Needed to Retain Information? (Option A)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 50)  # Just show first 50 components
plt.tight_layout()
plt.show()

# Find how many components for 90% and 95% variance
n_90 = np.argmax(cumsum_var >= 0.90) + 1
n_95 = np.argmax(cumsum_var >= 0.95) + 1

print(f"\nTo retain 90% of variance: {n_90} components needed")
print(f"To retain 95% of variance: {n_95} components needed")
print(f"We used only 2 components, retaining {pca_targets.explained_variance_ratio_.sum()*100:.2f}% of variance")

**Critical insight for ethics:** When we visualize in 2D, we're discarding most of the information. What genes or relationships might be hidden in those lost dimensions?

## 7. Conclusions

### What We Learned

*Summarize your key findings here after completing the analysis*

### Technical Insights
- PCA vs t-SNE differences
- Regulatory targets vs network topology
- Information loss through dimensionality reduction

### Biological Insights
- Patterns in C. elegans gene regulation
- Hub genes vs peripheral genes
- Functional modules or regulatory programs

### Limitations
- What did we not capture?
- What assumptions did we make?
- What would we do differently with more time/resources?